In [2]:
from watermark_REPEAT_NO_CONTEXT_BERT import watermark_model as repeat_no_context_bert_model
import json
import os
from utils1.preprocess_text import preprocess_txt
from tqdm import tqdm


def count_bit_result(text, result_file_1, result_file_0, model):
    sentences = model.sent_tokenize(text)
    sentences = [s for s in sentences if s.strip()]
    bit1 = []
    bit0 = []
    sentences = sentences[:2]
    print(len(sentences))
    for sent in sentences:
        tokens = model.tokenizer.tokenize(sent)
        greenlist_ids = model.get_greenlist_ids_bert(sent)
        for masked_token_index in range(0, len(tokens)):
            # print(f'==={masked_token_index}===')
            if not model.pos_filter(tokens, masked_token_index, sent):
                continue
            binary_encoding = 1 if model.tokenizer.convert_tokens_to_ids(
                    tokens[masked_token_index]) in greenlist_ids else 0
            if binary_encoding == 1:
                bit1.append({
                    'word': tokens[masked_token_index],
                    'bit':1
                })
            else:
                bit0.append({
                    'word': tokens[masked_token_index],
                    'bit':0
                })
    os.makedirs(os.path.dirname(result_file_1), exist_ok=True)
    with open(result_file_1, 'w', encoding='utf-8') as f:
        json.dump(bit1, f, indent=4, ensure_ascii=False)
    
    os.makedirs(os.path.dirname(result_file_0), exist_ok=True)
    with open(result_file_0, 'w', encoding='utf-8') as f:
        json.dump(bit0, f, indent=4, ensure_ascii=False)
    
    return ''.join(sentences)

def count_bit(text, model):
    sentences = model.sent_tokenize(text)
    sentences = [s for s in sentences if s.strip()]
    sentences = sentences[:2]
    count0 = 0
    count1 = 0
    # print(len(sentences))
    for sent in sentences:
        tokens = model.tokenizer.tokenize(sent)
        greenlist_ids = model.get_greenlist_ids_bert(sent)
        for masked_token_index in range(0, len(tokens)):
            # print(f'==={masked_token_index}===')
            try:
                if not model.pos_filter(tokens, masked_token_index, sent):
                    continue
                binary_encoding = 1 if model.tokenizer.convert_tokens_to_ids(
                        tokens[masked_token_index]) in greenlist_ids else 0
                if binary_encoding == 1:
                    count1 += 1
                else:
                    count0 += 1
            except BaseException as e:
                print(f'e=>{e}')
    
    return count0, count1

dataset = json.load(open('../result/REPEAT/water/HC3-random-800.json'))
model = repeat_no_context_bert_model(transform_model_path='../models/transform_model-bert_large-DISTANCE-CONDITION.pth')
    
count = []

for index in tqdm(range(len(dataset))):
    original_text = preprocess_txt(dataset[index]['original_text'])
    watermark_text = preprocess_txt(dataset[index]['watermark_text'])
    original_count0, original_count1 = count_bit(original_text, model)
    watermark_count0, watermark_count1 = count_bit(watermark_text, model)
    discharge = watermark_count1-watermark_count0
    count.append({
        'index': index,
        'count0': watermark_count0,
        'count1': watermark_count1,
        'discharge': discharge
    })
    
max_item = sorted(count, key=lambda x: x['discharge'])

BertForMaskedLM has generative capabilities, as `prepare_inputs_for_generation` is explicitly overwritten. However, it doesn't directly inherit from `GenerationMixin`. From 👉v4.50👈 onwards, `PreTrainedModel` will NOT inherit from `GenerationMixin`, and this model will lose the ability to call `generate` and other related functions.
  - If you're using `trust_remote_code=True`, you can get rid of this warning by loading the model with an auto class. See https://huggingface.co/docs/transformers/en/model_doc/auto#auto-classes
  - If you are the owner of the model architecture code, please modify your model class such that it inherits from `GenerationMixin` (after `PreTrainedModel`, otherwise you'll get an exception).
  - If you are not the owner of the model architecture class, please contact the model code owner to update it.
/root/miniconda3/lib/python3.12/site-packages/transformers/generation/configuration_utils.py:818: UserWarning: `return_dict_in_generate` is NOT set to `True`, but `

===bert-base-cased 完毕===


Some weights of the model checkpoint at /root/autodl-tmp/roberta-large-mnli were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


===roberta-large-mnli 完毕===
===roberta-large完毕===
===glove 完毕===
===nltk完毕===


/root/TextRepeat/watermark_REPEAT_NO_CONTEXT_BERT.py:144: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  transform_model.load_state_dict(torch.load(self.transform_path, map_l

...

 27%|██▋       | 218/800 [00:37<01:38,  5.94it/s]

e=>list index out of range


 61%|██████    | 487/800 [01:27<01:45,  2.97it/s]

e=>list index out of range
e=>list index out of range


100%|██████████| 800/800 [02:19<00:00,  5.73it/s]


In [11]:
max_item[29]

{'index': 34, 'count0': 3, 'count1': 7, 'discharge': 4}